# Spanish Socialist vs. People's Parties Affinity Classification

## VGG16 Fine Tuning

In the previous notebook ([TransferLearning.ipynb](TransferLearning.ipynb)) I tested the VGG16 transfer learning by training only the last FC layer. All the other convolutions blocks had the weights from the pre-trained VGG16.

This notebook, I will try to apply a fine tuning: to train 1 or 2 convolutional blocks + FC layer. The FC layer will use initial weights from the best model obtained in the previous step (Transfer Learning notebook). See more details at [Keras blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).

Let's load some libraries:

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

from numpy.random import seed
from tensorflow import set_random_seed
import time, os
import numpy as np
import keras

from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.metrics import roc_auc_score
from __future__ import with_statement
%matplotlib inline

c:\program files\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Definition of paths for dataset, previous trained weights for the FC layer, earlystopping model, etc.:

In [2]:
# Folder to save the models
modelFolder = 'saved_models'

# Path to the file with the weights of the pre-trained VGG16 model
weights_path = 'nets/vgg16_weights.h5'

# Path to the previous saved top model weights (FC layer trained in Transfer Learning notebook)
top_model_weights_path = os.path.join(modelFolder,'transferVGG16_bottleneck_fc_model.h5')

# Earlystoping saved model - this name will be modified later by including parameter values
earlystoping_path = './saved_models/fineTunning_earlystopnning.h5'

# Dimensions of our images
img_width, img_height = 150, 150

# Train & validation folders
train_data_dir      = 'data_politics/train'
validation_data_dir = 'data_politics/validation'

# Train parameters
nb_train_samples      = 80 # number of samples for training
nb_validation_samples = 20 # number of samples for validation
epochs = 10
batch_size = 2

Definition of the function that will do a fine tuning of the pre-trained VGG16 using FC layer weights trained in the previous notebook:
* Load the pre-trained VGG16 as the lower model,
* Add the top model as a FC layer,
* Load the previous calculated weights for the FC layer,
* Freeze a number of layers (a specific number of convolutional blocks): to freeze the last Conv block, freeze 15 layers; to freeze 2 last conv blocks, freeze only 11 layers.
* Compilte the computational graph of the model,
* Generate training & validation datasets from folders using data augmentation,
* Use earlystopping if the validation accuracy is not increasing in 10 iterations,
* Save the last best model,
* Use SGD optimizer,
* Search the best model using different values for the main hyperparameters: epochs, batch size, learning rate, momentum, and the number of layers to freeze.

See more details at [Keras blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).

In the first step, I will try one set of parameters:

In [3]:
def FineTunningVGG(epochs, batch_size, learning, mom, freezeLayers):
    # Fine tuning function using VGG16 and our weights for the FC layer (top model)
    
    # Set seeeds for reproductibility
    #seed(1)            # numpy seed
    #set_random_seed(2) # tensorflow seed
    
    # Build the VGG16 block using our input size 150, 150, 3
    base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

    # Build a classifier model to put on top of the convolutional model (FC layer / top model)
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))

    # Is necessary to start with a fully-trained classifier, including the top classifier,
    # in order to successfully do fine-tuning
    
    # Load the previous calculated weight for the top model
    top_model.load_weights(top_model_weights_path)

    # Add the model on top of the convolutional base
    model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

    # Set the first 'freezeLayers' layers to non-trainable (weights will not be updated)
    # This number depends on the blocks to freeze: for the last Conv block freeze 15 layers,
    # to freeze 2 last conv blocks freeze only 11 layers.
    for layer in model.layers[:freezeLayers]:
        layer.trainable = False

    # Compile the model with a SGD/momentum optimizer and a very slow learning rate.
    model.compile(loss='binary_crossentropy',
                  optimizer= optimizers.SGD(lr=learning, momentum=mom), # lr=1e-4, momentum=0.9
                  metrics=['accuracy'])

    # Prepare data augmentation configuration
    train_datagen = ImageDataGenerator(
        rescale = 1. / 255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True,
        rotation_range = 90)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    # Generate training and validation data
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

    # Start timer
    start_time = time.time()

    # Use earlystopping:
    callbacks=[EarlyStopping(
                            monitor='val_acc', 
                            patience=50, # very patient!
                            mode='max',
                            verbose=1),
                ModelCheckpoint(earlystoping_path[:-3]+'_e'+str(epochs)+'b'+str(batch_size)+'l'+str(learning)+'m'+str(mom)+'f'+str(freezeLayers)+'.h5',
                            monitor='val_acc', 
                            save_best_only=True, 
                            mode='max',
                            verbose=0)]

    # Fine-tune the model
    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        workers=7, # 7 cores of the CPU!
        verbose = 0,
        callbacks=callbacks) # remove this param if you dont need early stopping

    # Print training time
    print("Training time: %0.1f mins ---" % ((time.time() - start_time)/60))

    # Evaluate final test loss and accuracy scores
    scoresVal = model.evaluate_generator(validation_generator, nb_validation_samples//batch_size, workers=7)
    scoresTr  = model.evaluate_generator(train_generator, nb_train_samples//batch_size, workers=7)
    # Print the results
    print(freezeLayers, learning, mom, epochs, batch_size, scoresTr[0], scoresVal[0], scoresTr[1], scoresVal[1])

    # clean some memory
    del base_model
    del top_model
    del model

    del train_datagen
    del train_generator
    del validation_generator
    
    return

### Last Conv block + FC training

Let's try the fine tuning for FC and only the last Conv block using `SGD` and earlystopping:

In [4]:
# FineTunningVGG(epochs, batch_size, learning, mom, freezeLayers)
FineTunningVGG(200, 2, 1e-4,  0.9, 15)

Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00057: early stopping
Training time: 0.7 mins ---
15 0.0001 0.9 200 2 0.22392664093640632 0.5221619199961424 0.9 0.8


Even after 200 epochs, the model is underfitter (*validation ACC 80%*, *training ACC 90%*). We could decrease the drop rate to reduce the overfitting but we are using the same top model for loading the weights.

In the second step, let's try to use different paramters. You should use more values! With this function, you can search for several hyperparamters:

In [5]:
# Start total timer
start_time = time.time()

# Change your hyperparamters to search for
freezeLayersValues = [15] # 15 = freeze last Conv block, 11 = freeze last 2 Conv blocks
learningValues = [1e-4, 5e-4, 1e-3]
monValues = [0.8, 0.9]
epochsValues = [200]
batch_sizeValues = [2, 5]

# Print a header for results
print('Freeze', 'Learning', 'Momentum', 'epochs', 'batch_size', 'Loss_Tr', 'Loos_Val', 'Acc_Tr', 'Acc_Val')
for freezeLayers in freezeLayersValues: # 
    for learning in learningValues:
        for mom in monValues:
            for iepochs in epochsValues:
                for ibatch_size in batch_sizeValues:
                    try:
                        # Try to execute the fine tuning function
                        FineTunningVGG(iepochs, ibatch_size, learning, mom, freezeLayers)
                    except:
                        # If any error
                        print('==> Error:', freezeLayers, learning, mom, iepochs, ibatch_size)

# Print total execution time
print("Total time: %0.1f mins ---" % ((time.time() - start_time)/60))

Freeze Learning Momentum epochs batch_size Loss_Tr Loos_Val Acc_Tr Acc_Val
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00052: early stopping
Training time: 0.6 mins ---
15 0.0001 0.8 200 2 0.22526256359415128 0.623651000787504 0.875 0.65
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00051: early stopping
Training time: 0.5 mins ---
15 0.0001 0.8 200 5 0.25028148060664535 0.5772825814783573 0.9500000029802322 0.7000000067055225
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00058: early stopping
Training time: 0.7 mins ---
15 0.0001 0.9 200 2 0.1271283920003043 0.6267548841657117 0.9375 0.75
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00078: early stopping
Training time: 0.7 mins ---
15 0.0001 0.9 200 5 0.06473202907363884 0.7456640899181366 1.0 0.600000012665987
Found 80 images belonging to 2 classes.
Found 20 images belongin

Thus, by trainin the last Conv block and FC layer, you can obtain a `validation accuracy of 85%` in only 1 minute!

### Last 2 Conv block + FC training

Let's see what ACC we could obtain if we train the last 2 Conv blocks:

In [8]:
FineTunningVGG(200, 2, 0.001,  0.9, 11)

Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00060: early stopping
Training time: 0.9 mins ---
11 0.001 0.9 200 2 0.6931447878479957 0.6931476593017578 0.5 0.5


Let's check different parameters:

In [10]:
# Start total timer
start_time = time.time()

# Change your hyperparamters to search for
freezeLayersValues = [11] # 15 = freeze last Conv block, 11 = freeze last 2 Conv blocks
learningValues = [1e-4, 5e-4]
monValues = [0.8, 0.9]
epochsValues = [200]
batch_sizeValues = [2, 5]

# Print a header for results
print('Freeze', 'Learning', 'Momentum', 'epochs', 'batch_size', 'Loss_Tr', 'Loos_Val', 'Acc_Tr', 'Acc_Val')
for freezeLayers in freezeLayersValues: # 
    for learning in learningValues:
        for mom in monValues:
            for iepochs in epochsValues:
                for ibatch_size in batch_sizeValues:
                    try:
                        # Try to execute the fine tuning function
                        FineTunningVGG(iepochs, ibatch_size, learning, mom, freezeLayers)
                    except:
                        # If any error
                        print('==> Error:', freezeLayers, learning, mom, iepochs, ibatch_size)

# Print total execution time
print("Total time: %0.1f mins ---" % ((time.time() - start_time)/60))

Freeze Learning Momentum epochs batch_size Loss_Tr Loos_Val Acc_Tr Acc_Val
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00095: early stopping
Training time: 1.4 mins ---
11 0.0001 0.8 200 2 0.04930473282502135 0.7669441649690271 0.975 0.6
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00114: early stopping
Training time: 1.2 mins ---
11 0.0001 0.8 200 5 0.08175505104009062 0.792159415781498 0.9625000022351742 0.6500000134110451
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00111: early stopping
Training time: 1.7 mins ---
11 0.0001 0.9 200 2 0.3877230196038909 0.8754680104553699 0.85 0.7
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00095: early stopping
Training time: 1.0 mins ---
11 0.0001 0.9 200 5 0.0306460354840965 0.4901685491204262 0.9750000014901161 0.7500000149011612
Found 80 images belonging to 2 classes.
Found 20 imag

The accuracy of 80% is not better, so let's try a more flexible training by adding an extra FC layer (no weights loading!:

In [15]:
def FineTunningVGG2FC(epochs, batch_size, learning, mom, freezeLayers, drop, neurons):
    # Fine tuning function using VGG16 and our weights for the FC layer (top model)
    # 2 FC: 1 = 256 (as orginal), 2 = variable
    
    # Set seeeds for reproductibility
    #seed(1)            # numpy seed
    #set_random_seed(2) # tensorflow seed
    
    # Build the VGG16 block using our input size 150, 150, 3
    base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

    # Build a classifier model to put on top of the convolutional model (FC layer / top model)
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(drop))
    top_model.add(Dense(neurons, activation='relu'))
    top_model.add(Dropout(drop))
    top_model.add(Dense(1, activation='sigmoid'))

    # Is necessary to start with a fully-trained classifier, including the top classifier,
    # in order to successfully do fine-tuning
    
    # Load the previous calculated weight for the top model
    # top_model.load_weights(top_model_weights_path) # NO weights loading

    # Add the model on top of the convolutional base
    model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

    # Set the first 'freezeLayers' layers to non-trainable (weights will not be updated)
    # This number depends on the blocks to freeze: for the last Conv block freeze 15 layers,
    # to freeze 2 last conv blocks freeze only 11 layers.
    for layer in model.layers[:freezeLayers]:
        layer.trainable = False

    # Compile the model with a SGD/momentum optimizer and a very slow learning rate.
    model.compile(loss='binary_crossentropy',
                  optimizer= optimizers.SGD(lr=learning, momentum=mom), # lr=1e-4, momentum=0.9
                  metrics=['accuracy'])

    # Prepare data augmentation configuration
    train_datagen = ImageDataGenerator(
        rescale = 1. / 255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True,
        rotation_range = 90)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    # Generate training and validation data
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

    # Start timer
    start_time = time.time()

    # Use earlystopping:
    callbacks=[EarlyStopping(
                            monitor='val_acc', 
                            patience=100, # very patient!
                            mode='max',
                            verbose=1),
                ModelCheckpoint(earlystoping_path[:-3]+'_e'+str(epochs)+'b'+str(batch_size)+'l'+str(learning)+'m'+str(mom)+'f'+str(freezeLayers)+'d'+str(drop)+'n'+str(neurons)+'.h5',
                            monitor='val_acc', 
                            save_best_only=True, 
                            mode='max',
                            verbose=0)]

    # Fine-tune the model
    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        workers=7, # 7 cores of the CPU!
        verbose = 0,
        callbacks=callbacks) # remove this param if you dont need early stopping

    # Print training time
    print("Training time: %0.1f mins ---" % ((time.time() - start_time)/60))

    # Evaluate final test loss and accuracy scores
    scoresVal = model.evaluate_generator(validation_generator, nb_validation_samples//batch_size, workers=7)
    scoresTr  = model.evaluate_generator(train_generator, nb_train_samples//batch_size, workers=7)
    # Print the results
    print(freezeLayers, learning, mom, epochs, batch_size, drop, neurons, scoresTr[0], scoresVal[0], scoresTr[1], scoresVal[1])

    # clean some memory
    del base_model
    del top_model
    del model

    del train_datagen
    del train_generator
    del validation_generator
    
    return

In [16]:
# FineTunningVGG2FC(epochs, batch_size, learning, mom, freezeLayers, drop, neurons)
# 2 FC, 1 = 256, 2 = variable
# 11 0.0005 0.9 200 5 0.2282433467953524 0.5949657279998064 0.9250000044703484 0.800000011920929
for idrop in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8 , 0.9]:
    for ineurons in [128, 256, 512, 1024]:
        FineTunningVGG2FC(500, 5, 0.0005,  0.9, 11, idrop, ineurons)
        

Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00140: early stopping
Training time: 1.6 mins ---
11 0.0005 0.9 500 5 0.1 128 0.012667301005421905 0.8595507107675076 1.0 0.6500000134110451
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00334: early stopping
Training time: 3.7 mins ---
11 0.0005 0.9 500 5 0.1 256 0.00014107400495788625 0.5074687451124191 1.0 0.800000011920929
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00166: early stopping
Training time: 1.9 mins ---
11 0.0005 0.9 500 5 0.1 512 0.0030618831838182814 0.630945498123765 1.0 0.7500000149011612
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00122: early stopping
Training time: 1.4 mins ---
11 0.0005 0.9 500 5 0.1 1024 0.0073986175393656595 0.7015315694734454 1.0 0.7500000149011612
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 001

Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00102: early stopping
Training time: 1.3 mins ---
11 0.0005 0.9 500 5 0.9 1024 0.69315180554986 0.6931518018245697 0.5000000121071935 0.5000000149011612


The best results for 2 FC layers have been obtained for:
11 0.0005 0.9 500 5 0.4 256 0.09429500678379554 0.628852779045701 0.9500000029802322 0.8500000089406967

Let's try only 1 FC layer:

In [9]:
def FineTunningVGG1FC(epochs, batch_size, learning, mom, freezeLayers, drop, neurons):
    # Fine tuning function using VGG16 and our weights for the FC layer (top model)
    # 1 variable FC
    
    # Set seeeds for reproductibility
    #seed(1)            # numpy seed
    #set_random_seed(2) # tensorflow seed
    
    # Build the VGG16 block using our input size 150, 150, 3
    base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

    # Build a classifier model to put on top of the convolutional model (FC layer / top model)
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))

    top_model.add(Dense(neurons, activation='relu'))
    top_model.add(Dropout(drop))
    top_model.add(Dense(1, activation='sigmoid'))

    # Is necessary to start with a fully-trained classifier, including the top classifier,
    # in order to successfully do fine-tuning
    
    # Load the previous calculated weight for the top model
    # top_model.load_weights(top_model_weights_path) # NO weights loading

    # Add the model on top of the convolutional base
    model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

    # Set the first 'freezeLayers' layers to non-trainable (weights will not be updated)
    # This number depends on the blocks to freeze: for the last Conv block freeze 15 layers,
    # to freeze 2 last conv blocks freeze only 11 layers.
    for layer in model.layers[:freezeLayers]:
        layer.trainable = False

    # Compile the model with a SGD/momentum optimizer and a very slow learning rate.
    model.compile(loss='binary_crossentropy',
                  optimizer= optimizers.SGD(lr=learning, momentum=mom), # lr=1e-4, momentum=0.9
                  metrics=['accuracy'])

    # Prepare data augmentation configuration
    train_datagen = ImageDataGenerator(
        rescale = 1. / 255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True,
        rotation_range = 90)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    # Generate training and validation data
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

    # Start timer
    start_time = time.time()

    # Use earlystopping:
    callbacks=[EarlyStopping(
                            monitor='val_acc', 
                            patience=100, # very patient!
                            mode='max',
                            verbose=1),
                ModelCheckpoint(earlystoping_path[:-3]+'_e'+str(epochs)+'b'+str(batch_size)+'l'+str(learning)+'m'+str(mom)+'f'+str(freezeLayers)+'d'+str(drop)+'n'+str(neurons)+'.h5',
                            monitor='val_acc', 
                            save_best_only=True, 
                            mode='max',
                            verbose=0)]

    # Fine-tune the model
    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        workers=7, # 7 cores of the CPU!
        verbose = 0,
        callbacks=callbacks) # remove this param if you dont need early stopping

    # Print training time
    print("Training time: %0.1f mins ---" % ((time.time() - start_time)/60))

    # Evaluate final test loss and accuracy scores
    scoresVal = model.evaluate_generator(validation_generator, nb_validation_samples//batch_size, workers=7)
    scoresTr  = model.evaluate_generator(train_generator, nb_train_samples//batch_size, workers=7)
    # Print the results
    print(freezeLayers, learning, mom, epochs, batch_size, drop, neurons, scoresTr[0], scoresVal[0], scoresTr[1], scoresVal[1])

    # clean some memory
    del base_model
    del top_model
    del model

    del train_datagen
    del train_generator
    del validation_generator
    
    return

In [5]:
# if we use Adam => 0.5 ACC!
for idrop in [0.1, 0.5, 0.9]:
    for ineurons in [128, 256, 512, 1024]:
        FineTunningVGG1FC(500, 5, 0.0005,  0.9, 11, idrop, ineurons) # 1 FC no weight loading

Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00125: early stopping
Training time: 1.4 mins ---
11 0.0005 0.9 500 5 0.1 128 0.05436937647755258 0.46945618093013763 0.9875000007450581 0.800000011920929
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00217: early stopping
Training time: 2.4 mins ---
11 0.0005 0.9 500 5 0.1 256 0.013837655367296975 1.8047126233577728 1.0 0.6000000163912773
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00116: early stopping
Training time: 1.3 mins ---
11 0.0005 0.9 500 5 0.1 512 0.3622851693071425 0.5248122736811638 0.8250000104308128 0.800000011920929
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 00211: early stopping
Training time: 2.3 mins ---
11 0.0005 0.9 500 5 0.1 1024 0.0007533760493174668 0.9735153184155934 1.0 0.800000011920929
Found 80 images belonging to 2 classes.
Found 20 images belonging

Best results for 1 FC have been obtained for:
11 0.0005 0.9 500 5 0.1 512 0.3622851693071425 0.5248122736811638 0.8250000104308128 0.800000011920929

## Conclusion

* If you apply the fine tuning for the last conv block of VGG16 + FC (top model), you can obtain an accuracy of `75%`. This values is no better compare with the small CNN and the Transfer Learning results (`over 80%`).
* But if you train the 2 last Conv block of VGG16 + FC (top model), you can obtain test accuracy of `85%`!
* The search space was limited and possible additional hyperparameter combinations should be tested including drop rate, optimizer or the base model (not only VGG16, it could be Inception, RNN, etc.).

If you need a classifier to detect Spanish political affinity using people portraits, you should try the VGG16 fine tuning for at least 85% of accuracy.

Have fun with DL!
@muntisa

### Acknowledgements

I gratefully acknowledge the support of NVIDIA Corporation with the donation of the Titan Xp GPU used for this research ([https://developer.nvidia.com/academic_gpu_seeding](https://developer.nvidia.com/academic_gpu_seeding)).